In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal

In [ ]:
import datetime
from datetime import datetime as datetime_1

In [ ]:
np.random.seed(1)

df_bellingcat = pd.read_excel("../../DataSources/Bellingcat_Conversations_Type_war_CodingNew.xlsx",dtype = {'conversation_id': str,"id":str})
df_bellingcat
print(df_bellingcat.head())

In [ ]:
df_bellingcat = df_bellingcat[df_bellingcat['date']>=datetime.datetime(year=2021,month=11,day=16)]
df_bellingcat = df_bellingcat[df_bellingcat['lang']=="en"]
print(df_bellingcat.tail())

In [ ]:
df_bellingcat['year'] = df_bellingcat.date.dt.year
df_bellingcat['month'] = df_bellingcat.date.dt.month
print(df_bellingcat.head())

In [ ]:
df_bellingcat["Final Type"]=df_bellingcat["Final Type"].replace("Publication of content regarding bellingcat", "Bellingcat Ops")
df_bellingcat["Final Type"]=df_bellingcat["Final Type"].replace("Crowdsource", "Crowdsourcing")
df_bellingcat["Final Type"]=df_bellingcat["Final Type"].replace("Share other content", "Other Promotion")
df_bellingcat["Final Type"]=df_bellingcat["Final Type"].replace("Tool presentation and training", "Tools and Training")

## Followers

In [ ]:
df_followers= pd.read_excel("../../DataSources/Followers_final.xlsx")
df_followers = df_followers[["Date","followers"]].groupby([pd.Grouper(freq="M",key="Date")])["followers"].max().reset_index()
print(df_followers.head())

In [ ]:
df_followers['year'] = df_followers.Date.dt.year
df_followers['month'] = df_followers.Date.dt.month
print(df_followers.head())

In [ ]:
df_bellingcat=df_bellingcat.merge(df_followers, on=["year","month"], how='left').drop(columns=['month', 'year', 'Date'])

## Engagement depending on the type of content 

In [ ]:
df_bellingcat["total_engagement"] = df_bellingcat["likes"]+df_bellingcat["replies"]+df_bellingcat["quotes"]+df_bellingcat["retweets"]
df_bellingcat["total_engagement_per_follower"] = df_bellingcat["total_engagement"] / df_bellingcat["followers"]
df_bellingcat['media_photo_Binary'] = np.where(df_bellingcat['media_photo'] > 0, 1, 0)
df_bellingcat['media_videos_Binary'] = np.where(df_bellingcat['media_videos'] > 0, 1, 0)
df_bellingcat['media_animated_gif_Binary'] = np.where(df_bellingcat['media_animated_gif'] > 0, 1, 0)
df_bellingcat['media_review_image_url_Binary'] = np.where(df_bellingcat['media_review_image_url'] > 0, 1, 0)
df_bellingcat["media_Binary"] = np.where((  (df_bellingcat['media_photo'] > 0) | (df_bellingcat['media_videos'] > 0)), "Media", "No Media")
df_bellingcat["media_Binary_numeric"] = np.where(((df_bellingcat['media_photo'] > 0) | (df_bellingcat['media_videos'] > 0)), 1, 0)
df_bellingcat["total_media"] =   df_bellingcat['media_photo'] + df_bellingcat['media_videos']
df_bellingcat['urls_Binary'] = np.where(df_bellingcat['urls'] > 0, 1, 0)
df_bellingcat['hashtags_Binary'] = np.where(df_bellingcat['hashtags'] > 0, 1, 0)
df_bellingcat['call_to_action'] = np.where(df_bellingcat['Final Type'] == "Crowdsource", 1, 0)
df_bellingcat['log_engagement_per_post_per_follower']=np.log(df_bellingcat["total_engagement_per_follower"]/df_bellingcat["count"])
df_bellingcat["likes_log"]=np.log(((df_bellingcat["likes"]+1)/ df_bellingcat["followers"])/df_bellingcat["count"])
df_bellingcat["conversation_binary"]=np.where(df_bellingcat['count'] > 1, "Thread", "Single")
df_bellingcat["replies_log"]=np.log(((df_bellingcat["replies"]+1)/ df_bellingcat["followers"])/df_bellingcat["count"])
df_bellingcat["quotes_log"]=np.log(((df_bellingcat["quotes"]+1)/ df_bellingcat["followers"])/df_bellingcat["count"])
df_bellingcat["retweets_log"]=np.log(((df_bellingcat["retweets"]+1)/ df_bellingcat["followers"])/df_bellingcat["count"])
df_bellingcat["normlog_engagement_per_follower"]=np.log(df_bellingcat["total_engagement_per_follower"])

In [ ]:
df_correlation =df_bellingcat[["likes","replies","quotes","retweets"]]
df_correlation.corr()

In [ ]:
df_bellingcat_not_filtered = df_bellingcat.copy()

In [ ]:
df_bellingcat = df_bellingcat[df_bellingcat["Final Type"] != "Publication of other events"]

In [ ]:
df_bellingcat['quantiles'] = (
    pd.qcut(df_bellingcat['log_engagement_per_post_per_follower'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
)

In [ ]:
df_bellingcat['Final Type'].value_counts()

Distribution of total engagement based on the type of content of the post. 

In [ ]:
df_bellingcat.to_csv("../../DataSources/Dataset_Graphs/RQ2_Content_Engagement/first_graphs.csv")
df_bellingcat.to_excel("../../DataSources/Dataset_Graphs/RQ2_Content_Engagement/engagement_for_sentiment.xlsx")

Distribution of Log of total engagement based on the type of content 

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for type in df_bellingcat["Final Type"].unique():
    fig.add_trace(go.Box(#x=df_bellingcat['Final Type'][df_bellingcat['Final Type'] == type],
                            y=df_bellingcat['log_engagement_per_post_per_follower'][df_bellingcat['Final Type'] == type],
                            name=type,
                            #box_visible=True,
                            #meanline_visible=True
                            
                             ))

fig.update_layout(
        width=800, height=600, yaxis_title = "Log10 Engagement",legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.8
    )  )                      
fig.show()

In [ ]:
kruskal(df_bellingcat[df_bellingcat["Final Type"]=="Self-Promotion"]['log_engagement_per_post_per_follower'],df_bellingcat[df_bellingcat["Final Type"]=="Research Publication"]['log_engagement_per_post_per_follower'],df_bellingcat[df_bellingcat["Final Type"]=="Bellingcat Ops"]['log_engagement_per_post_per_follower'],df_bellingcat[df_bellingcat["Final Type"]=="Crowdsourcing"]['log_engagement_per_post_per_follower'],df_bellingcat[df_bellingcat["Final Type"]=="Tools and Training"]['log_engagement_per_post_per_follower'],df_bellingcat[df_bellingcat["Final Type"]=="Other Promotion"]['log_engagement_per_post_per_follower'])

In [ ]:
fig = px.box(df_bellingcat[df_bellingcat["Final Type"]=="Research Publication"],
                            y='log_engagement_per_post_per_follower',
                            color="media_Binary",
                            #box_visible=True,
                            #meanline_visible=True
                            title="Distribuiton of log engagement by type of media "
                             )

fig.show()

In [ ]:
x = df_bellingcat[df_bellingcat['media_Binary']=="No Media"]["log_engagement_per_post_per_follower"]
y = df_bellingcat[df_bellingcat['media_Binary']=="Media"]["log_engagement_per_post_per_follower"]

U1, p = mannwhitneyu(x, y, method="auto",alternative="less")
print(U1,p)

In [ ]:
fig = px.box(df_bellingcat,x="Final Type",
                            y='log_engagement_per_post_per_follower',
                            color="media_Binary",
                            #box_visible=True,
                            #meanline_visible=True
                            title="Distribuiton of log engagement by type of media "
                             )

fig.show()

In [ ]:
list = df_bellingcat['Final Type'].unique()

for type in list: 
    x = df_bellingcat[(df_bellingcat['Final Type']==type) & (df_bellingcat['media_Binary']=="No Media")]["log_engagement_per_post_per_follower"]
    y = df_bellingcat[(df_bellingcat['Final Type']==type) & (df_bellingcat['media_Binary']=="Media")]["log_engagement_per_post_per_follower"]
    
    U1, p = mannwhitneyu(x, y, method="auto",alternative="less")
    print(type,U1,p)

In [ ]:
def f(x):
    return pd.Series(dict(rate_media =  x['media_Binary_numeric'].sum()/(x['count'].count()),
                        rate_hashtags = x['hashtags_Binary'].sum()/(x['count'].count()),
                        rate_urls = x['urls_Binary'].sum()/(x['count'].count()), 
                        amount_tweet =  x['count'].sum(),
                        amount_conversation =  x['count'].count(),
                        conversation_depth =  x['count'].sum()/x['count'].count()))

df_bellingcat_Rates = df_bellingcat.groupby(["Final Type"]).apply(f).reset_index()

In [ ]:
print(df_bellingcat_Rates)

In [ ]:
df_bellingcat_Rates=df_bellingcat_Rates.sort_values(by=['amount_tweet'], ascending=False)
df_bellingcat_Rates.to_csv("../../DataSources/Dataset_Graphs/RQ2_Content_Engagement/rates.csv", index = False)

In [ ]:
def f(x):
    return pd.Series(dict(total_media =  x['total_media'].sum(),
                        total_hashtags = x['hashtags'].sum(),
                        total_urls = x['urls'].sum(), 
                        amount_tweet =  x['count'].sum(),
                        amount_conversation =  x['count'].count()))

df_bellingcat_totals = df_bellingcat.groupby(["Final Type"]).apply(f).reset_index()

In [ ]:
print(df_bellingcat_totals)

In [ ]:
df_bellingcat_totals.to_csv("../../DataSources/Dataset_Graphs/RQ2_Content_Engagement/total_media.csv", index= False)

## Changes in engagement after and before the war

In [ ]:
df_bellingcat["time_frame"]= np.where(df_bellingcat['date']>=datetime.datetime(year=2022,month=2,day=24),"after","before")
print(df_bellingcat.head())
df_bellingcat_not_filtered["time_frame"]= np.where(df_bellingcat_not_filtered['date']>=datetime.datetime(year=2022,month=2,day=24),"after","before")
print(df_bellingcat.head())

Change in distribution of engagement based on content

There is a change in engagement in almost all categories except tool presentation

In [ ]:
fig = px.box(df_bellingcat,
                            y="log_engagement_per_post_per_follower",
                            color="time_frame",
                            #box_visible=True,
                            #meanline_visible=True
                            title="Change in engagement before and after the war per follower",
                            category_orders={"time_frame": ["before", "after"],}
                             )

fig.show()

In [ ]:
x = df_bellingcat[df_bellingcat['time_frame']=="before"]["log_engagement_per_post_per_follower"]
y = df_bellingcat[df_bellingcat['time_frame']=="after"]["log_engagement_per_post_per_follower"]

U1, p = mannwhitneyu(x, y, method="auto",alternative="less")
print(U1,p)

In [ ]:
fig = px.box(df_bellingcat,x="Final Type",
                            y='log_engagement_per_post_per_follower',
                            color="time_frame",
                            #box_visible=True,
                            #meanline_visible=True
                            #title="Change in engagement before and after the war per follower",
                            category_orders={"time_frame": ["before", "after"],},
                            labels={"normlog_engagement_per_follower":"Log Scale Engagment per Follower",
                            "time_frame":"Timeframe","Final Type":"f"},
                            width=800, height=600
                             )

#fig.update_xaxes(categoryorder='array', categoryarray= ['Research Publication', "Self-Promotion",'Tool Presentation', 'Crowdsource', 'Support other content', 'Publication of event regarding bellingcat'])

fig.show()

In [ ]:
list = df_bellingcat['Final Type'].unique()

for type in list: 
    x = df_bellingcat[(df_bellingcat['Final Type']==type) & (df_bellingcat['time_frame']=="before")]["log_engagement_per_post_per_follower"]
    y = df_bellingcat[(df_bellingcat['Final Type']==type) & (df_bellingcat['time_frame']=="after")]["log_engagement_per_post_per_follower"]
    
    U1, p = mannwhitneyu(x, y, method="auto",alternative="less")
    print(type,U1,p)

In [ ]:
df_bellingcat.to_csv("../../DataSources/Dataset_Graphs/RQ3_Content_Engagement/before_after.csv", index=False)

In [ ]:
df_bellingcat_per_follower= df_bellingcat[["Final Type","time_frame","log_engagement_per_post_per_follower"]].groupby(by=["time_frame","Final Type"])["log_engagement_per_post_per_follower"].median().reset_index()
print(df_bellingcat_per_follower)

In [ ]:
df_bellingcat_per_follower_overall= df_bellingcat[["time_frame","total_engagement_per_follower"]].groupby(by=["time_frame"])["total_engagement_per_follower"].median().reset_index()
print(df_bellingcat_per_follower_overall)
df_bellingcat_per_follower_overall["Final Type"]=["overall","overall"]
print(df_bellingcat_per_follower_overall)

In [ ]:
df_bellingcat_per_follower_total= pd.concat([df_bellingcat_per_follower,df_bellingcat_per_follower_overall])
print(df_bellingcat_per_follower_total)

In [ ]:
df_bellingcat["month"] = df_bellingcat["date"].dt.month

In [ ]:
df_bellingcat_volumen_after_war= df_bellingcat[["Final Type","time_frame","count"]].groupby(by=["time_frame","Final Type"])["count"].count().reset_index()
print(df_bellingcat_volumen_after_war)

In [ ]:
percentage_change = df_bellingcat_volumen_after_war.sort_values(['time_frame'], inplace = False, ascending=[False]).groupby(by=["Final Type"])["count"].apply(pd.Series.pct_change) 
percentage_change

In [ ]:
percentage_relative_purpose_afterwar = df_bellingcat_volumen_after_war.copy()
percentage_relative_purpose_afterwar["relative"] = df_bellingcat_volumen_after_war["count"]/df_bellingcat_volumen_after_war.groupby(["time_frame"])["count"].transform("sum")
percentage_relative_purpose_afterwar

In [ ]:
df_bellingcat_volumen_after_war.to_csv("../../DataSources/Dataset_Graphs/RQ3_Content_Engagement/df_bellingcat_volumen_after_war.csv", index=False)

In [ ]:
df_bellingcat_volumen_after_war_eng = df_bellingcat[["Final Type","time_frame","total_engagement"]].groupby(by=["time_frame","Final Type"])["total_engagement"].sum().reset_index()
print(df_bellingcat_volumen_after_war_eng)

## Analysis of replies content 

In [ ]:
def f(x):
    return pd.Series(dict(replies = x['replies'].sum(), 
                        quotes = x['quotes'].sum(), 
                        retweets = x['retweets'].sum(), 
                        likes = x['likes'].sum(), 
                        media_photo = x['media_photo'].sum(), 
                        media_videos = x['media_videos'].sum(), 
                        media_animated_gif = x['media_animated_gif'].sum(), 
                        media_review_image_url = x['media_animated_gif'].sum(), 
                        hashtags = x['hashtags'].sum(), 
                        urls = x['urls'].sum(), 
                        count = x['count'].sum(), 
                        text =  ', '.join(x['text'])))
 
df_bellingcat_filtered_grouped = df_bellingcat.groupby(["Final Type","time_frame"]).apply(f).reset_index()
print(df_bellingcat_filtered_grouped.head())

In [ ]:
df_replies = pd.read_excel("../../DataSources/replies_with_media_war.xlsx",dtype = {'conversation_id': str,"id":str})


def date_convert(date_to_convert):
    return datetime_1.strptime(date_to_convert ,"%Y-%m-%dT%H:%M:%S.%fZ")

    
df_replies['date'] = df_replies["date"].apply(date_convert) #.dt.date()
df_replies=df_replies.sort_values(by=['date'], ascending=True)
df_replies['date'] = df_replies["date"].dt.date
df_replies = df_replies[df_replies["author_id"] != "2315512764"]
df_replies = df_replies[df_replies["author_id"] != 2315512764]
df_replies = df_replies[df_replies["lang"] == "en"]

def f(x):
    return pd.Series(dict(replies_reply = x['replies'].sum(), 
                        quotes_reply = x['quotes'].sum(), 
                        retweets_reply = x['retweets'].sum(), 
                        likes_reply = x['likes'].sum(), 
                        media_photo_reply = x['media_photo'].sum(), 
                        media_videos_reply = x['media_videos'].sum(), 
                        media_animated_gif_reply = x['media_animated_gif'].sum(), 
                        media_review_image_url_reply = x['media_animated_gif'].sum(), 
                        hashtags_reply = x['hashtags'].sum(), 
                        urls_reply = x['urls'].sum(), 
                        count_reply = x['id'].count(), 
                        text_reply =  ', '.join(x['text'])))


#print(df_replies.head())
#print(df_replies.columns)
df_replies = df_replies.groupby(['conversation_id']).apply(f).reset_index()
print(df_replies.head())

In [ ]:
df_bellingcat_replies=df_bellingcat_not_filtered.merge(df_replies, on='conversation_id', how='left')
print(df_bellingcat_replies.head())


In [ ]:
df_bellingcat_replies.to_excel("../../DataSources/Dataset_Graphs/RQ3_Content_Engagement/df_bellingcat_replies_v2.xlsx")

In [ ]:
df_summarized_replies = df_bellingcat_replies.groupby(by=["Final Type","time_frame"]).sum().reset_index()
print(df_summarized_replies.head())

In [ ]:
df_bellingcat_replies["total_media_reply"] = df_bellingcat_replies['media_photo_reply'] +df_bellingcat_replies['media_videos_reply'] 
df_bellingcat_replies["media_Binary_reply"] = np.where(( (df_bellingcat_replies['media_photo_reply'] > 0) | (df_bellingcat_replies['media_videos_reply'] > 0)), "Media", "No Media")
df_summarized_replies = df_bellingcat_replies.groupby(by=["conversation_id","time_frame"]).sum().reset_index()

In [ ]:
def f(x):
    return pd.Series(dict(
                        total_media_reply = x['total_media_reply'].sum()/x['count_reply'].sum(), 
                        hashtags_reply = x['hashtags_reply'].sum()/x['count_reply'].sum(), 
                        urls_reply = x['urls_reply'].sum()/x['count_reply'].sum(), 
                        average_reply = x['count_reply'].mean(),
                        total_replies = x['count_reply'].sum(),
                        conversations = x['conversation_id'].count()))


df_summarized_replies_average = df_summarized_replies.groupby(['time_frame']).apply(f).reset_index()
print(df_summarized_replies_average.head())